https://wikidocs.net/158023  
https://wikidocs.net/157896

## 챗봇 데이터 셋을 이용

In [1]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

토크나이저에서 사용되는 스페셜 토큰들을 정의

In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [3]:
import urllib.request
'''
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
'''
Chatbot_Data = pd.read_csv("ChatBotData1.csv")
print(Chatbot_Data.shape)
# Test 용으로 300개 데이터만 처리한다.
#Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

(107, 3)


,Q,A,label
0,화남,분노의 엉덩이댄스!!,3.0
1,화남,화날땐 부리부리~,3.0
2,화남,화날땐 하나둘셋 야!,3.0
3,화남,우이씨,3.0
4,화남,뚜씨 뚜씨,3.0


In [4]:
BOS = "</s>"
EOS = "</s>"
PAD = "<pad>"
MASK = "<unused0>"

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [6]:
# 배치 데이터를 만들기 위해
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

Dataset, DataLoader 정의

In [7]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [8]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [9]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    #print("token_ids ====> ", token_ids)
    #print("mask =====> ", mask)
    #print("label =====> ", label)
print("end")

start
end


C:\Users\admin\AppData\Local\Temp\ipykernel_20688\384841629.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


---

### 실제 모델을 학습

In [10]:
import torch

In [11]:
torch.__version__

'1.11.0'

In [12]:
import pytorch_lightning

In [13]:
pytorch_lightning.__version__

'1.8.2'

In [14]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [15]:
# 토크나이저에서 사용되는 스페셜 토큰들을 정의
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [16]:
# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 모델과 토크나이저
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [17]:
'''
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
'''
Chatbot_Data = pd.read_csv("ChatBotData1.csv")
# Test 용으로 300개 데이터만 처리한다.
#Chatbot_Data = Chatbot_Data[:300]
print(Chatbot_Data.shape)
Chatbot_Data.head()

(107, 3)


,Q,A,label
0,화남,분노의 엉덩이댄스!!,3.0
1,화남,화날땐 부리부리~,3.0
2,화남,화날땐 하나둘셋 야!,3.0
3,화남,우이씨,3.0
4,화남,뚜씨 뚜씨,3.0


https://github.com/songys/Chatbot_data  
데이터의 Q 필드를 발화, A 필드를 발화 그리고 감정 레이블을 로 매핑

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch)

In [19]:
device

device(type='cuda')

In [20]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [21]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

epoch = 20
Sneg = -1e18

In [22]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
count = 0
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        
        # tensor cuda로 올리기
        token_ids = token_ids.cuda()
        mask = mask.cuda()
        label = label.cuda()
    
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)    
        
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        
        # 학습 끝
        optimizer.step()
    count += 1
    print(f'epoch{count}의 loss : ', avg_loss) # CrossEntropyLoss

epoch1의 loss :  tensor(63.3086, device='cuda:0', grad_fn=<DivBackward0>)
epoch2의 loss :  tensor(66.9410, device='cuda:0', grad_fn=<DivBackward0>)
epoch3의 loss :  tensor(56.5796, device='cuda:0', grad_fn=<DivBackward0>)
epoch4의 loss :  tensor(54.1897, device='cuda:0', grad_fn=<DivBackward0>)
epoch5의 loss :  tensor(47.9178, device='cuda:0', grad_fn=<DivBackward0>)
epoch6의 loss :  tensor(54.5884, device='cuda:0', grad_fn=<DivBackward0>)
epoch7의 loss :  tensor(53.7012, device='cuda:0', grad_fn=<DivBackward0>)
epoch8의 loss :  tensor(54.5072, device='cuda:0', grad_fn=<DivBackward0>)
epoch9의 loss :  tensor(52.0489, device='cuda:0', grad_fn=<DivBackward0>)
epoch10의 loss :  tensor(52.0011, device='cuda:0', grad_fn=<DivBackward0>)
epoch11의 loss :  tensor(47.2882, device='cuda:0', grad_fn=<DivBackward0>)
epoch12의 loss :  tensor(61.1605, device='cuda:0', grad_fn=<DivBackward0>)
epoch13의 loss :  tensor(52.6227, device='cuda:0', grad_fn=<DivBackward0>)
epoch14의 loss :  tensor(57.2619, device='cuda:0

In [25]:
sent = "2" # 0=일상, 1=부정, 2=긍정

with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0)
            
            # tensor cuda로 올리기
            input_ids = input_ids.cuda()
            
            pred = model(input_ids)
            pred = pred.logits.detach().cpu()
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 화남
Chatbot > 화날땐 하나둘셋 야
user > 역겨움
Chatbot > 으익 맛없는 표정
user > 두려움
Chatbot > 도와줘ㅠㅠ
user > 행복
Chatbot > 행복행복 액션가면~
user > 슬픔
Chatbot > 슬픔의 눈물댄스~
user > 놀람
Chatbot > 하나둘셋야  놀랐지
user > 중립
Chatbot > 흐음
user > quit


In [26]:
# 학습한 모델 저장
torch.save(model, './trained_model.pt')